In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/IndNumSp')
from matplotlib import pyplot as plt


# Load the Haar cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the pre-trained mouth XML file for mouth detection
mouth_cascade = cv2.CascadeClassifier('/content/drive/MyDrive/IndNumSp/mouth.xml')

def extract_lip_features(frame, scale_factor):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Check if any faces are detected
    if len(faces) == 0:
        print("No faces found.")
        return None

    # Extract the region of interest (mouth) from each detected face
    for (x, y, w, h) in faces:
        # Adjust the region of interest to focus on the lower part of the face
        roi_y = int(y + h * 0.6)
        roi_h = int(h * 0.4)
        roi = gray[roi_y:roi_y+roi_h, x:x+w]

        # Detect mouths in the region of interest
        mouths = mouth_cascade.detectMultiScale(roi, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Check if any mouths are detected
        if len(mouths) > 0:
            # Extract the largest detected mouth as the lip region
            (mx, my, mw, mh) = max(mouths, key=lambda m: m[2] * m[3])
            lip_roi = roi[my:my+mh, mx:mx+mw]

            # Calculate the position and size for enlarging the lip region
            new_w = int(mw * scale_factor)
            new_h = int(mh * scale_factor)
            pos_x = x + mx + int((mw - new_w) / 2)
            pos_y = roi_y + my + int((mh - new_h) / 2)

            # Enlarge the lip region
            enlarged_lip_roi = cv2.resize(lip_roi, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

            # Create a copy of the frame and draw a rectangle around the lip region
            frame_with_lip = frame.copy()
            cv2.rectangle(frame_with_lip, (pos_x, pos_y), (pos_x + new_w, pos_y + new_h), (0, 255, 0), 2)

            # Display the frame with the lip region
            cv2_imshow(frame_with_lip)

            # Display the extracted lip region
            cv2_imshow(lip_roi)

            # Wait for a key press to proceed to the next frame
            cv2.waitKey(0)

In [ ]:


# Define the scale factor for enlarging the lip region
scale_factor = 1.5

# Open the video file
#video_path = '3_14.mp4'  
video_path = '6_12.mp4' #The Asian with Glasses
#video_path = '1_5.mp4' # Still behaves badly on the lips of the black persons
#video_path = '5_63.mp4' 
cap = cv2.VideoCapture(video_path)

# Check if the video file was opened successfully
if not cap.isOpened():
    print("Failed to open the video file.")

# Read and process each frame of the video
while True:
    # Read a frame from the video
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if not ret:
        print("Failed to read a frame from the video.")
        break

    # Extract the lip features from the frame
    extract_lip_features(frame, scale_factor)

# Release the video file
cap.release()

# Close all windows
cv2.destroyAllWindows()